# HR Analytics project

#### Import all necessary libraries:

In [62]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import random
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from statsmodels.graphics.gofplots import qqplot
%matplotlib inline
from matplotlib import pyplot

ModuleNotFoundError: No module named 'pandas_profiling'

In [63]:
# Load training dataset
Train = pd.read_csv("Train.csv")
Test = pd.read_csv("Test.csv")

In [64]:
# Delete ID from Train as this information is not required.
Train = Train.drop('ID',axis=1)

In [65]:
Train.head()

,Age,Gender,Marital_status,Num_comp_worked,Work_Experience,Last_salary,Salary_hike,Work_hours,Traveling_required,NoticePeriod_buyout,...,Annual_leave,Medical_Family,Child_day_care,Nursing_room,Parking,Comp_size,Comp_rating,Work_life,Stresslevel,Accept_offer
0,0,0,0,1,1,2,4,0,0,0,...,1,0,0,NaN,0,4,3,3,2,0
1,1,0,1,0,2,1,1,0,1,0,...,1,0,0,NaN,1,4,2,1,4,0
2,0,0,0,0,2,1,1,0,1,0,...,1,0,0,NaN,1,3,2,3,3,1
3,1,0,1,1,1,2,1,0,0,0,...,1,0,0,NaN,1,4,4,3,1,1
4,1,1,1,0,3,4,3,0,1,1,...,1,1,0,1.0,0,4,2,3,3,0


In [66]:
# Delete ID from Test as this information is not required.
Test = Test.drop('Id',axis=1)

In [67]:
pandas_profiling.ProfileReport(Train)

NameError: name 'pandas_profiling' is not defined

In [68]:
pandas_profiling.ProfileReport(Test)

NameError: name 'pandas_profiling' is not defined

EDA observations:

1. All 33 Variables are preprocessed into categorical in 2,3,4 dimensions.
2. Total 1712 (48.5%) are missing values observed in Nursing_room. Need to work on imputing missing values.
3. "Accept_offer" is the output variable with binary (0,1) values.

### We will try to impute missing values in "Nursing_room" with regression technique in Training and Test datasets

In [69]:
# Split Train data vertically into 2 datasets 1. With Null and 2. Without Null
data_with_null = Train[Train.isnull().any(axis=1)]
data_without_null = Train.dropna()

In [50]:
# Split Test data vertically into 2 datasets 1. With Null and 2. Without Null
Test_with_null = Test[Test.isnull().any(axis=1)]
Test_without_null = Test.dropna()

In [51]:
#All features except Nursing_room
train_data_x = data_without_null.iloc[:, data_without_null.columns != 'Nursing_room']
# Only Nursing_room
train_data_y = data_without_null.iloc[:, data_without_null.columns == 'Nursing_room']

In [52]:
#All features except Nursing_room
test_data_x = Test_without_null.iloc[:, Test_without_null.columns != 'Nursing_room']
# Only Nursing_room
test_data_y = Test_without_null.iloc[:, Test_without_null.columns == 'Nursing_room']

In [53]:
X_train0, X_test0, y_train0, y_test0 = train_test_split(Train.drop('Nursing_room',axis=1),Train['Nursing_room'], test_size=0.4852,random_state=101)
logreg = LogisticRegression()

NameError: name 'train_test_split' is not defined

In [54]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(Test.drop('Nursing_room',axis=1),Test['Nursing_room'], test_size=0.493,random_state=101)
logregT = LogisticRegression()

NameError: name 'train_test_split' is not defined

In [55]:
# Training with Train data
logreg.fit(train_data_x,train_data_y.values.ravel())

NameError: name 'logreg' is not defined

In [56]:
# Training with Test data
logregT.fit(test_data_x,test_data_y.values.ravel())

NameError: name 'logregT' is not defined

In [57]:
# Predict for the whole dataset and replace only the missing values later
test_data = data_with_null.iloc[:, data_with_null.columns != 'Nursing_room']
predicted = data_with_null
predicted['Nursing_room'] = pd.DataFrame(logreg.predict(test_data))
predicted.index = test_data.index

NameError: name 'logreg' is not defined

In [58]:
# Predict for the whole dataset and replace only the missing values later
test1_data = Test_with_null.iloc[:, Test_with_null.columns != 'Nursing_room']
Tpredicted = Test_with_null
Tpredicted['Nursing_room'] = pd.DataFrame(logregT.predict(test1_data))
Tpredicted.index = test1_data.index

NameError: name 'logregT' is not defined

In [59]:
# Replace all null values with new predicted values in Train_with_Null
#for i in range(len(data_with_null)):
data_with_null.Nursing_room = predicted

C:\Users\mozak\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [60]:
# Replace all null values with new predicted values in Test_with_Null
#for i in range(len(Test_with_null)):
Test_with_null.Nursing_room = Tpredicted

C:\Users\mozak\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [61]:
# Concat both datasets with null and without null values in Train: 
frames = [data_with_null, data_without_null]
Train1 = pd.concat(frames)
Train1.reset_index(inplace = True) 

In [23]:
# Concat both datasets with null and without null values in Test: 
Tframes = [Test_with_null, Test_without_null]
Test1 = pd.concat(Tframes)
Test1.reset_index(inplace = True)

In [24]:
Train1.isnull().values.sum()

0

In [25]:
Test1.isnull().values.sum()

0

### Finding the algorithm that best fits our requirement: 

In [26]:
# Value percentage of our target category
Train['Accept_offer'].value_counts(normalize=True) * 100

0    52.452509
1    47.547491
Name: Accept_offer, dtype: float64

In [27]:
# defining feature matrix(X) and response vector(y) 
X = Train1.iloc[:, Train1.columns != 'Accept_offer']
y = Train1.iloc[:, Train1.columns == 'Accept_offer'] 

In [28]:
seed = 10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

NameError: name 'train_test_split' is not defined

# Test options and evaluation metric
kfold = StratifiedKFold(n_splits=10, random_state=1)
cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')

models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1)
	cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

### Create the Model Evaluation Report

In [ ]:
def evaluation_report(model, X_train, Y_train, X_test, Y_test, prediction):
    
    #Print classification report
    print(classification_report(Y_test, prediction.round(), target_names=['Accept_offer_No', 'Accept_offer_Yes']))
    
    #Building confusion matrix
    confusion_m = confusion_matrix(Y_test, prediction.round())
    print(confusion_m)
    
    # rc = 'axes.facecolor':'#757575ff', 'figure.facecolor':'#757575ff',
    
    #Create the heatmap of confusion matrix
    plt.figure(figsize = (12,8))
    sns.set(rc={'axes.grid': False, 'axes.linewidth': 1 }, font_scale=1.5)
    sns.heatmap(confusion_m, 
            cmap='Purples_r', 
            annot=True, 
            linewidths=0.5)
    plt.title('{} Confusion Matrix for Test Set'.format(model))
    plt.xlabel('Predicted Class')
    plt.ylabel('Real Class')
    plt.show()
    
    scores = cross_val_score(model, X_train, Y_train, cv=10)
    print('Cross Validation Accuracy Scores: {:.3f}(+/- {:.2f})'.format(scores.mean(), scores.std()*2))

    scores_Test = cross_val_score(model, X_test, Y_test, cv=10)
    print('Cross Validation Accuracy Scores - Test Set: {:.3f}(+/- {:.2f})'.format(scores_Test.mean(), scores_Test.std()*2))

### Prediction with Logistic Regression:

In [29]:
# train the model using the training sets:
ModelLogreg = LogisticRegression()
ModelLogreg.fit(X_train, y_train.values.ravel())
LR_y_pred = ModelLogreg.predict(X_test)

NameError: name 'LogisticRegression' is not defined

In [30]:
LR_y_predicted

NameError: name 'LR_y_predicted' is not defined

In [31]:
# Making predictions on the new testing dataset:
LR_y_predicted['Accept_offer'] = ModelLogreg.predict(Test1)
#Val_Test_LR = Test1.iloc[:, Test1.columns == 'Accept_offer']
#Val_Test_LR['Accept_offer'] = ModelLogreg.predict(X_test)

NameError: name 'ModelLogreg' is not defined

In [32]:
# Merge with main dataset 
LR_Test = pd.merge(Test1, LR_y_predicted, how='inner', left_index=True, right_index=True)

NameError: name 'LR_y_predicted' is not defined

In [33]:
evaluation_report(ModelLogreg, X_train, y_train, X_test, y_test, LR_y_pred)

NameError: name 'evaluation_report' is not defined

In [34]:
pickle.dump(ModelLogreg, open('model.pkl','wb'))

model = pickle.load(open('model.pkl','rb'))

NameError: name 'pickle' is not defined

Conclusions:
1. With Logarithmic Regression we obtained 97% of accuracy and better precision and recall.

### Prediction with KNN Algorithm:

In [35]:
KNClassifier = KNeighborsClassifier(n_neighbors=1)
KNClassifier.fit(X_train, y_train.values.ravel())

NameError: name 'KNeighborsClassifier' is not defined

In [36]:
KNN_y_pred = KNClassifier.predict(X_test)

NameError: name 'KNClassifier' is not defined

In [37]:
# creating odd list of K for KNN
neighbors = list(range(1, 50, 2))

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train.values.ravel(), cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

NameError: name 'KNeighborsClassifier' is not defined

In [38]:
# changing to misclassification error
mse = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[mse.index(min(mse))]
print("The optimal number of neighbors is {}".format(optimal_k))

# plot misclassification error vs k
plt.plot(neighbors, mse)
plt.xlabel("Number of Neighbors K")
plt.ylabel("Misclassification Error")
plt.show()

ValueError: min() arg is an empty sequence

In [39]:
evaluation_report(KNClassifier, X_train, y_train, X_test, y_test, KNN_y_pred)

NameError: name 'evaluation_report' is not defined

Conclusions:
1. From the above graph, we can see that K = 1 is optimal with better accuracy of 61%.
2. Since the variable in consideraton are all categorical KNN algorithm accuracy is log as compared to other algorithm.

### Prediction with Decision Tree Algorithm:

In [40]:
DCClassifier = DecisionTreeClassifier()
DCClassifier.fit(X_train, y_train.values.ravel())
RF_y_pred = DCClassifier.predict(X_test)

NameError: name 'DecisionTreeClassifier' is not defined

In [41]:
evaluation_report(DCClassifier, X_train, y_train.values.ravel(), X_test, y_test, RF_y_pred)

NameError: name 'evaluation_report' is not defined

Conclusions:
1. The prediction accuracy is 100%.
2. Precision and Recall are also 100%.
3. By the results, the predictions appear to be more biased. So we shall consider Logistic regression as better option for out problem.